### Analysis: Startup Metrics Dashboard, FitLoop

Lila Weizer

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

#### I. Read in Synthetic Data

The python script used to generate this synthetic data can be found in /data within the project repository. 

In [9]:
users = pd.read_csv('../data/users.csv')
events = pd.read_csv('../data/events.csv')
payments = pd.read_csv('../data/payments.csv')

In [10]:
users.head()

,user_id,signup_date,acquisition_channel
0,10001,2025-05-15,Paid Ads
1,10002,2025-06-08,Referral
2,10003,2025-04-06,Paid Ads
3,10004,2025-03-09,Paid Ads
4,10005,2025-03-27,Paid Ads


In [11]:
events.head()

,user_id,event_type,timestamp
0,10001,share,2025-06-08 17:00:00
1,10001,login,2025-06-01 08:00:00
2,10001,browse,2025-06-25 20:00:00
3,10001,share,2025-06-26 15:00:00
4,10001,share,2025-05-16 07:00:00


In [12]:
payments.head()

,user_id,amount,payment_date,plan_type
0,16253,30,2025-06-09,Quarterly
1,14685,10,2025-05-25,Monthly
2,11732,10,2025-03-28,Monthly
3,14743,30,2025-03-03,Quarterly
4,14522,10,2025-05-04,Monthly


#### II. Wrangling and KPI Calculation

##### a.) DAU / MAU 

Daily active users (DAU) is a useful metric for measuring product stickiness and short-term user engagement. Similarly, monthly active users (MAU) represents a high-level view of user engagement over time. Therefore, it is valuable to calculate both metrics separately. 

In [26]:
events['date'] = pd.to_datetime(events['timestamp']).dt.date
dau = events.groupby('date')['user_id'].nunique().reset_index(name='DAU')
events['event_month'] = pd.to_datetime(events['date']).dt.to_period('M')
mau = events.groupby('event_month')['user_id'].nunique().reset_index(name='MAU')

In [27]:
dau

,date,DAU
0,2025-01-01,5
1,2025-01-02,10
2,2025-01-03,10
3,2025-01-04,15
4,2025-01-05,18
...,...,...
176,2025-06-26,1670
177,2025-06-27,1670
178,2025-06-28,1797
179,2025-06-29,1754


In [36]:
mau

,event_month,MAU
0,2025-01,951
1,2025-02,2296
2,2025-03,4038
3,2025-04,5724
4,2025-05,7677
5,2025-06,9359


In [45]:
def change(df):
    change = []
    for i in np.arange(0, len(df) -1):
        diff = (df[i+1] - df[i]) / df[i]
        change.append(diff)
    return change

MoM = change(mau['MAU'])
MoM.insert(0, 0)
mau['MoM Change'] = MoM
mau

,event_month,MAU,MoM Change
0,2025-01,951,0.000000
1,2025-02,2296,1.414301
2,2025-03,4038,0.758711
3,2025-04,5724,0.417533
4,2025-05,7677,0.341195
5,2025-06,9359,0.219096


I also wanted to prioritize caluclating the DAU / MAU ratio, which is another measure of stickiness that shows what percentage of monthly users engage on a typical day. For high performing apps of this type, we normally want to see 20-30%+ to indicate stickiness.

In [47]:
dau['month'] = pd.to_datetime(dau['date']).dt.to_period('M')
dau_mau = dau.merge(mau, left_on='month', right_on='event_month')
dau_mau['DAU_MAU_ratio'] = dau_mau['DAU'] / dau_mau['MAU']
dau_mau

,date,DAU,month,event_month,MAU,MoM Change,DAU_MAU_ratio
0,2025-01-01,5,2025-01,2025-01,951,0.000000,0.005258
1,2025-01-02,10,2025-01,2025-01,951,0.000000,0.010515
2,2025-01-03,10,2025-01,2025-01,951,0.000000,0.010515
3,2025-01-04,15,2025-01,2025-01,951,0.000000,0.015773
4,2025-01-05,18,2025-01,2025-01,951,0.000000,0.018927
...,...,...,...,...,...,...,...
176,2025-06-26,1670,2025-06,2025-06,9359,0.219096,0.178438
177,2025-06-27,1670,2025-06,2025-06,9359,0.219096,0.178438
178,2025-06-28,1797,2025-06,2025-06,9359,0.219096,0.192008
179,2025-06-29,1754,2025-06,2025-06,9359,0.219096,0.187413


In [48]:
print("The average DAU/MAU for the five-month period is ", np.mean(dau_mau['DAU_MAU_ratio']))

The average DAU/MAU for the five-month period is  0.08397146290014636


##### b.) Retention (Day 1, Day 7, and Day 30)